# Implement MathChat with the Agent framework

Based on the current agent, we can implement the MathChat framework directly. MathChat is a convesational framework for math problem solving, and a user proxy agent is used to enable an automatic coversation with the LLM assistant. The current agent framework clearly defines the assistant agent and the user proxy agent, and further allows human inputs. By inheriting the `UserProxyAgent` class, we can implement the original user proxy agent in MathChat, which we name it `MathUserProxyAgent` under the current framework. Throught investiagting in several problems, we find this new implementation of MathChat is **the same** as that in the original MathChat framework. 
The original implementation of MathChat is in this [branch](https://github.com/kevin666aa/FLAML/tree/gpt_math_solver/flaml/autogen/math), and you can find more details in this [paper](https://arxiv.org/abs/2306.01337).


## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
# %pip install flaml[autogen]

## Set your API Endpoint

The [`config_list_gpt4_gpt35`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_gpt4_gpt35) function tries to create a list of gpt-4 and gpt-3.5 configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.


In [1]:
from flaml import oai

config_list = config_list = [{
    'model' : 'gpt-4',
    'api_key': open("key_e.txt").read().strip(),
}]


/home/ykw5399/.conda/envs/flaml_dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # only if OpenAI API key is found
    {
        'model': 'gpt-4',
        'api_key': '<your first Azure OpenAI API key here>',
        'api_base': '<your first Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the at least one Azure OpenAI API key is found
    {
        'model': 'gpt-4',
        'api_key': '<your second Azure OpenAI API key here>',
        'api_base': '<your second Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-03-15-preview',
    },  # only if the second Azure OpenAI API key is found
]
```

You can directly override it if the above function returns an empty list, i.e., it doesn't find the keys in the specified locations.

## Reproduce MathChat with the Agent framework

1. Following MathChat, we set the system message of the assistant agent to ` "You are a helpful assistant"`, and the actual prompt will be concatentated in the user message.
2. We will using the `MathUserProxyAgent` to handle the autoreplies. `MathUserProxyAgent` inherits the `UserProxyAgent` in this agent framework, and modifies the `autoreply` for query handling. The human_input_mode should be set to `Never` since MathChat is a fully automatic system. We add a `reset` function to clear cached variables from solving one math problem.

In [6]:
from flaml.autogen.agent.assistant_agent import AssistantAgent
from flaml.autogen.agent.math_user_proxy_agent import MathUserProxyAgent
from flaml.autogen.math_utils import get_answer

oai.ChatCompletion.start_logging()

# create an assistant which is essentially a AssistantAgent instance named "coding_agent"
assistant = AssistantAgent("assistant", 
                           system_message="You are a helpful assistant.",
                           request_timeout=600, 
                           seed=42, 
                           config_list=config_list)
# create a UserProxyAgent instance named "user"
mathproxyagent = MathUserProxyAgent(human_input_mode="NEVER")
# the assistant receives a message from the user, which contains the task description


default_prompt = """
Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answer, put the answer in \\boxed{}.

Problem: """



### Example 1

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

Solution: 
We have \begin{align*} (2x+10)(x+3)&<(3x+9)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)&<3(x+3)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)-3(x+3)(x+8)&<0 \quad \Rightarrow
\\ (2x+10-(3x+24))(x+3)&<0 \quad \Rightarrow
\\ (-x-14)(x+3)&<0 \quad \Rightarrow
\\ (x+14)(x+3)&>0.
\end{align*} This inequality is satisfied if and only if $(x+14)$ and $(x+3)$ are either both positive or both negative.  Both factors are positive for $x>-3$ and both factors are negative for $x<-14$.  When $-14<x<-3$, one factor is positive and the other negative, so their product is negative.   Therefore, the range of $x$ that satisfies the inequality is $ \boxed{(-\infty, -14)\cup(-3,\infty)} $.

In [36]:
assistant.reset()
mathproxyagent.reset()

math_problem =  "Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."
assistant.receive(
    default_prompt+math_problem,
    mathproxyagent,
)


**** assistant received message from MathChatAgent ****


Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query

### Example 2
Problem: For what negative value of $k$ is there exactly one solution to the system of equations \begin{align*}
y &= 2x^2 + kx + 6 \\
y &= -x + 4?
\end{align*}

Solution: Setting the two expressions for $y$ equal to each other, it follows that $2x^2 + kx + 6 = -x + 4$. Re-arranging, $2x^2 + (k+1)x + 2 = 0$. For there to be exactly one solution for $x$, then the discriminant of the given quadratic must be equal to zero. Thus, $(k+1)^2 - 4 \cdot 2 \cdot 2 = (k+1)^2 - 16 = 0$, so $k+1 = \pm 4$. Taking the negative value, $k = \boxed{-5}$.

In [27]:
assistant.reset()
mathproxyagent.reset()

math_problem =  "For what negative value of $k$ is there exactly one solution to the system of equations \\begin{align*}\ny &= 2x^2 + kx + 6 \\\\\ny &= -x + 4?\n\\end{align*}"
assistant.receive(
    default_prompt+math_problem,
    mathproxyagent,
)


**** assistant received message from MathChatAgent ****


Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query

### Example 3
Problem: Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas.

Solution: For the roots to be real and rational, the discriminant must be a perfect square. Therefore, $(-7)^2-4 \cdot 1 \cdot c = 49-4c$ must be a perfect square. The only positive perfect squares less than 49 are $1$, $4$, $9$, $16$, $25$, and $36$. The perfect squares that give a integer value of $c$ are $1$, $9$, and $25$. Thus, we have the equations $49-4c=1$, $49-4c=9$, and $49-4c=25$. Solving, we get that the positive integer values of c are $\boxed{12, 10, 6}$.

In [30]:
assistant.reset()
mathproxyagent.reset()

math_problem =  "Find all positive integer values of $c$ such that the equation $x^2-7x+c=0$ only has roots that are real and rational. Express them in decreasing order, separated by commas."
assistant.receive(
    default_prompt+math_problem,
    mathproxyagent,
)


**** assistant received message from MathChatAgent ****


Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

You may choose from three ways to solve the problem:
Case 1: If the problem can be solved with Python code directly, please write a program to solve it. You can enumerate all possible arrangements if needed.
Case 2: If the problem cannot be handled in the above two ways, please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query

### Example 4: Using other prompts

MathChat allows different prompts that instruct assistant to solve the problem. In the paper, two more prompts are used: one is a simplied prompt from the default prompt that uses Python only, the other prompt further allows the use of Wolfram Alpha based on this simplied prompt. Here we give an example of using the simplied Python prompt: 

In [7]:
assistant.reset()
mathproxyagent.reset()

# Note: prompt changed:
customized_prompt = """Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answer, put the answer in \\boxed{}.

Problem: """

math_problem =  "Find all numbers $a$ for which the graph of $y=x^2+a$ and the graph of $y=ax$ intersect. Express your answer in interval notation."
assistant.receive(
    customized_prompt+math_problem,
    mathproxyagent,
)




**** assistant received message from MathChatAgent ****

Let's use Python to solve a math problem.

Query requirements:
You should always use the 'print' function for the output and use fractions/radical forms instead of decimals.
You can use packages like sympy to help you.
You must follow the formats below to write your code:
```python
# your code
```

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answer, put the answer in \boxed{}.

Problem: Find all numbers $a$ for which the graph of $y=x^2+a$ and the graph of $y=ax$ intersect. Express your answer in interval notation.

**** MathChatAgent received messa